In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/NLP/validation")  #imput the path of the validation data here
print(os.getcwd())

/content/drive/MyDrive/NLP/validation


# Installing Necessary Packages

In [ ]:
!pip install pytorch_pretrained_bert
!pip install pandas
!pip install nltk
!pip install rouge
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Ignore all UserWarnings
warnings.filterwarnings("ignore", message=".*DeprecationWarning.*") # Ignore warnings containing "DeprecationWarning"

# Preprocessing Reports

**Importing necessary libraries**

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Loading annual reports**

In [ ]:
import pandas as pd
import os
folder_path = 'annual_reports'
data = []
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as f:
            text = f.read()
        data.append({'filename': filename, 'text': text})
df = pd.DataFrame(data)

In [ ]:
df

,filename,text
0,30777.txt,25695 19 March 2018 3:29 PM Proof 7 256...
1,30778.txt,Staffline Group plc \nAnnual Report 2017\n En...
2,30779.txt,"National Express Group PLC, Annual Report 201..."
3,30781.txt,Annual Report and Accounts 2017\nCorero Netwo...
4,30782.txt,THE GLOBAL TQA EXPERTS\nANNUAL REPORT 201 7 O...
...,...,...
358,33130.txt,2002\nAnnual Report& Accounts DICOM Group p...
359,33098.txt,2014 ANNUAL REPORT Contents\nStrategic Report...
360,33097.txt,Retail \nspecialists\nANNUAL REPORT 2014\nHA...
361,33154.txt,Annual Report and Accounts\nDixons Carphone p...


**Replace line breaks and redundant spacings**

In [ ]:
def replace_newlines(df):
    df["text"] = df["text"].str.replace("\n", " ")
    df["text"] = df["text"].str.replace(r"\s+", " ")
    df.loc[:,'text'] = df['text'].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    return df

In [ ]:
df = replace_newlines(df)

<ipython-input-9-7dfbab7689df>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r"\s+", " ")
<ipython-input-9-7dfbab7689df>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:,'text'] = df['text'].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()


In [ ]:
df

,filename,text
0,30777.txt,25695 19 March 2018 3:29 PM Proof 7 25695 19 M...
1,30778.txt,Staffline Group plc Annual Report 2017 Ena bl ...
2,30779.txt,"National Express Group PLC, Annual Report 2017..."
3,30781.txt,Annual Report and Accounts 2017 Corero Network...
4,30782.txt,THE GLOBAL TQA EXPERTS ANNUAL REPORT 201 7 OUR...
...,...,...
358,33130.txt,2002 Annual Report& Accounts DICOM Group plc L...
359,33098.txt,2014 ANNUAL REPORT Contents Strategic Report 4...
360,33097.txt,Retail specialists ANNUAL REPORT 2014 HAMMERSO...
361,33154.txt,Annual Report and Accounts Dixons Carphone plc...


**Take first 10% out**

In [ ]:
def take_first_ten_precent(df):
    for i in range(len(df)):
        text_length = len(df.loc[i, "text"])
        split_index = int(text_length * 0.1)
        df.loc[i, "text"] = df.loc[i, "text"][:split_index]  # Take first 10% of text
    return df

In [ ]:
df = take_first_ten_precent(df)

In [ ]:
print(df.iloc[0]['text'].__len__()) # To print the length of the text document

46220


**Lowercasing**

In [ ]:
def lower_case_text(df):
    df['text'] = df['text'].str.lower()
    return df

In [ ]:
df = lower_case_text(df)

In [ ]:
df

,filename,text
0,30777.txt,25695 19 march 2018 3:29 pm proof 7 25695 19 m...
1,30778.txt,staffline group plc annual report 2017 ena bl ...
2,30779.txt,"national express group plc, annual report 2017..."
3,30781.txt,annual report and accounts 2017 corero network...
4,30782.txt,the global tqa experts annual report 201 7 our...
...,...,...
358,33130.txt,2002 annual report& accounts dicom group plc l...
359,33098.txt,2014 annual report contents strategic report 4...
360,33097.txt,retail specialists annual report 2014 hammerso...
361,33154.txt,annual report and accounts dixons carphone plc...


**Remove specific words**

In [ ]:
def remove_specific_words(input_string, words_to_remove):
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words_to_remove) + r')\b'
    cleaned_string = re.sub(pattern, '', input_string)
    return cleaned_string

words_to_remove = ["a", "an", "at", "the", "only"]
df["text"] = df["text"].apply(remove_specific_words, args=(words_to_remove,))  # Remove specific words

In [ ]:
df

,filename,text
0,30777.txt,25695 19 march 2018 3:29 pm proof 7 25695 19 m...
1,30778.txt,staffline group plc annual report 2017 ena bl ...
2,30779.txt,"national express group plc, annual report 2017..."
3,30781.txt,annual report and accounts 2017 corero network...
4,30782.txt,global tqa experts annual report 201 7 our hi...
...,...,...
358,33130.txt,2002 annual report& accounts dicom group plc l...
359,33098.txt,2014 annual report contents strategic report 4...
360,33097.txt,retail specialists annual report 2014 hammerso...
361,33154.txt,annual report and accounts dixons carphone plc...


# Summarization

**Importing necessary libraries**

In [ ]:
from nltk.tokenize import sent_tokenize
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

**Generating Sentence Embeddings using BERT**

In [ ]:
def bertSent_embeding(sentences):


    """
    Input a list of sentence tokens

    Output a list of latent vectors, each vector is a sentence representation

    Bert Base Uncased model produce 12 layers of latent vector

    """

    ## Add sentence head and tail as BERT requested
    marked_sent = ["[CLS] " +item + " [SEP]" for item in sentences]

    ## USE Bert tokenizization
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_sent = [tokenizer.tokenize(item) for item in marked_sent]

    ## index to BERT vocabulary
    indexed_tokens = [tokenizer.convert_tokens_to_ids(item) for item in tokenized_sent]
    tokens_tensor = [torch.tensor([item]) for item in indexed_tokens]

    ## add segment id as BERT requested
    segments_ids = [[1] * len(item) for ind,item in enumerate(tokenized_sent)]
    segments_tensors = [torch.tensor([item]) for item in segments_ids]

     ## load BERT base model i.e. BERT BASE UNCASED MODEL and set to evaluation mode
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    bert_model.eval()

    ## Output 12 layers of latent vector
    assert len(tokens_tensor) == len(segments_tensors)
    encoded_layers_list = []
    for i in range(len(tokens_tensor)):
        with torch.no_grad():
            encoded_layers, _ = bert_model(tokens_tensor[i], segments_tensors[i])
        encoded_layers_list.append(encoded_layers)

    ## Use only the last layer vector
    token_vecs_list = [layers[11][0] for layers in encoded_layers_list]

    ## Pooling word vector to sentence vector,Here we used mean pooling.
    sentence_embedding_list = [torch.mean(vec, dim=0).numpy() for vec in token_vecs_list]
    return sentence_embedding_list

**K Means clustering algorithm for clustering sentences to form the summary**

In [ ]:
def kmeans_sumIndex(sentence_embedding_list):

    """
    Input a list of embeded sentence vectors

    Output an list of indices of sentence in the paragraph, represent the clustering of key sentences

    """

    ## Here We used dynamic method to assign the number of clusters
    n_clusters = np.ceil(len(sentence_embedding_list)**0.5)

    kmeans = KMeans(n_clusters=int(n_clusters))
    kmeans = kmeans.fit(sentence_embedding_list)
    sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
    sum_index = sorted(sum_index)
    return sum_index

**Main function which does the summarization**

In [ ]:
def bertSummarize(text, max_sequence_length=512):

    """
    Input a paragraph as string

    Output the summary including a few key sentences using BERT sentence embedding and clustering

    """
    sentences = sent_tokenize(text)
    sentences = [sentence for sentence in sentences if len(sentence) <= max_sequence_length]
    sentence_embedding_list = bertSent_embeding(sentences)
    sum_index = kmeans_sumIndex(sentence_embedding_list)
    summary = ' '.join([sentences[ind] for ind in sum_index])
    return summary

**Validation Dataset dataframe on which summarization will be performed**

In [ ]:
df

,filename,text
0,30777.txt,25695 19 march 2018 3:29 pm proof 7 25695 19 m...
1,30778.txt,staffline group plc annual report 2017 ena bl ...
2,30779.txt,"national express group plc, annual report 2017..."
3,30781.txt,annual report and accounts 2017 corero network...
4,30782.txt,global tqa experts annual report 201 7 our hi...
...,...,...
358,33130.txt,2002 annual report& accounts dicom group plc l...
359,33098.txt,2014 annual report contents strategic report 4...
360,33097.txt,retail specialists annual report 2014 hammerso...
361,33154.txt,annual report and accounts dixons carphone plc...


**Calling the summarization method and creating .txt files for each summary generated and saving in a directory**

In [ ]:
import os
import pandas as pd

def summarize_and_save(row, text_column="text", filename_column="filename"):
    """Summarizes the text from a dataframe row and saves it with the specified filename."""

    summary = bertSummarize(row[text_column])  # Assume bertSummarize function defined elsewhere
    filename = row[filename_column]
    summary_filename = f"{filename[:-4]}_summary.txt"

    os.makedirs("gen_sum", exist_ok=True)
    with open(f"gen_sum/{summary_filename}", "w") as f:
        f.write(summary)

# Example usage with dataframe:
for i, row in df.iterrows():
    summarize_and_save(row)  # Pass the entire row to the function

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# Evaluation Metrics

**Loading the summary dataframe**

In [ ]:
import pandas as pd
import os
folder_path = 'gen_sum'
data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as f:
            text = f.read()
        data.append({'filename': filename, 'summary': text})

summary_frame = pd.DataFrame(data)

In [ ]:
summary_frame

,filename,summary
0,30777_summary.txt,bodycote lives by culture of honest and trans...
1,30778_summary.txt,jobs. company overview staffline is leading w...
2,30779_summary.txt,board believes that this gives more comparabl...
3,30781_summary.txt,dir ec t ors c onsider ebitd t o be bet ter ...
4,30782_summary.txt,that drives ultimate intertek differentiator ...
...,...,...
358,33130_summary.txt,its customers are supported by more than some ...
359,33098_summary.txt,we look for investments that are priced attrac...
360,33097_summary.txt,"centre will provide 23,800m 2 of retailing and..."
361,33154_summary.txt,"we have set ourselves ambitious goals, not fi..."


In [ ]:
print(summary_frame.iloc[0]['summary'].__len__()) # To print the length of the generated summary

2542


**Load the gold summaries against which generated summaries will be compared**

In [ ]:
import pandas as pd
import os
folder_path = 'gold_summaries'
data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as f:
            text = f.read()
        data.append({'filename': filename, 'gold_summary': text})

gold_summaries = pd.DataFrame(data)

In [ ]:
gold_summaries

,filename,gold_summary
0,31013_1.txt,STRATEGIC REPORT\n6 Robert Walters plc Annua...
1,31013_2.txt,STRATEGIC REPORT\n2 Robert Walters plc Annua...
2,31014_1.txt,S TR A TE G I C R E P O R T \nCHIEF EXECUTIVE...
3,31014_2.txt,GR O U P O V E RV I E W\nANGLO PACIFIC AT A G...
4,31014_3.txt,WE ENTER 2018 IN A STRONG FINANCIAL \nPOSITIO...
...,...,...
1245,31008_1.txt,Chief executive’s review \n12 Rathbone Brothe...
1246,31012_2.txt,24%\n18%\n27%\n31%\nTT Electronics at a glanc...
1247,31012_1.txt,Chief Executive’s strategic review\nA year of...
1248,31012_3.txt,Chairman’s statement\nA pivotal year \nfor TT...


**Concatinating both the generated summary and the gold summary frame**

In [ ]:
df_sorted = gold_summaries.sort_values(by='filename')
summary_frame['base_filename'] = summary_frame['filename'].str.split('_').str[0]
df_sorted['base_filename'] = df_sorted['filename'].str.split('_').str[0]
merged_df = pd.merge(summary_frame, df_sorted, on='base_filename', how='inner')
result_df = merged_df[['filename_x', 'filename_y', 'summary', 'gold_summary']]

In [ ]:
result_df

,filename_x,filename_y,summary,gold_summary
0,30777_summary.txt,30777_1.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\n02...
1,30777_summary.txt,30777_2.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nFi...
2,30777_summary.txt,30777_3.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nA....
3,30778_summary.txt,30778_1.txt,jobs. company overview staffline is leading w...,10\nStaffline Group plc Annual Report 2017\nC...
4,30778_summary.txt,30778_2.txt,jobs. company overview staffline is leading w...,3 Overview Strategic Report Corporate Governa...
...,...,...,...,...
1245,33154_summary.txt,33154_3.txt,"we have set ourselves ambitious goals, not fi...",Dixons Carphone plc Annual Report and Accoun...
1246,33155_summary.txt,33155_1.txt,this is where we are differentiated by our emb...,"IN CONVERSATION WITH \nDAVID ATKINS, CHIEF EX..."
1247,33155_summary.txt,33155_2.txt,this is where we are differentiated by our emb...,HIGHLIGHTS\n2015 overview\nPortfolio value\n(...
1248,33155_summary.txt,33155_3.txt,this is where we are differentiated by our emb...,Setting Hammerson’s \nvalues and standards\nC...


In [ ]:
result_df = result_df.rename(columns={"filename_x": "generated_summary_file", "filename_y": "gold_summary_file"})

In [ ]:
result_df

,generated_summary_file,gold_summary_file,summary,gold_summary
0,30777_summary.txt,30777_1.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\n02...
1,30777_summary.txt,30777_2.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nFi...
2,30777_summary.txt,30777_3.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nA....
3,30778_summary.txt,30778_1.txt,jobs. company overview staffline is leading w...,10\nStaffline Group plc Annual Report 2017\nC...
4,30778_summary.txt,30778_2.txt,jobs. company overview staffline is leading w...,3 Overview Strategic Report Corporate Governa...
...,...,...,...,...
1245,33154_summary.txt,33154_3.txt,"we have set ourselves ambitious goals, not fi...",Dixons Carphone plc Annual Report and Accoun...
1246,33155_summary.txt,33155_1.txt,this is where we are differentiated by our emb...,"IN CONVERSATION WITH \nDAVID ATKINS, CHIEF EX..."
1247,33155_summary.txt,33155_2.txt,this is where we are differentiated by our emb...,HIGHLIGHTS\n2015 overview\nPortfolio value\n(...
1248,33155_summary.txt,33155_3.txt,this is where we are differentiated by our emb...,Setting Hammerson’s \nvalues and standards\nC...


**Function to calculate the Rouge Score**

In [ ]:
from rouge import Rouge
rouge = Rouge()

def calculate_rouge(row):
    scores = rouge.get_scores(row['summary'], row['gold_summary'])[0]
    return scores

result_df['rouge_scores'] = result_df.apply(calculate_rouge, axis=1)

In [ ]:
result_df

,generated_summary_file,gold_summary_file,summary,gold_summary,rouge_scores
0,30777_summary.txt,30777_1.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\n02...,"{'rouge-1': {'r': 0.1966205837173579, 'p': 0.5..."
1,30777_summary.txt,30777_2.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nFi...,"{'rouge-1': {'r': 0.2653061224489796, 'p': 0.2..."
2,30777_summary.txt,30777_3.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nA....,"{'rouge-1': {'r': 0.22871046228710462, 'p': 0...."
3,30778_summary.txt,30778_1.txt,jobs. company overview staffline is leading w...,10\nStaffline Group plc Annual Report 2017\nC...,"{'rouge-1': {'r': 0.09648466716529544, 'p': 0...."
4,30778_summary.txt,30778_2.txt,jobs. company overview staffline is leading w...,3 Overview Strategic Report Corporate Governa...,"{'rouge-1': {'r': 0.18396226415094338, 'p': 0...."
...,...,...,...,...,...
1245,33154_summary.txt,33154_3.txt,"we have set ourselves ambitious goals, not fi...",Dixons Carphone plc Annual Report and Accoun...,"{'rouge-1': {'r': 0.2727272727272727, 'p': 0.2..."
1246,33155_summary.txt,33155_1.txt,this is where we are differentiated by our emb...,"IN CONVERSATION WITH \nDAVID ATKINS, CHIEF EX...","{'rouge-1': {'r': 0.18683001531393567, 'p': 0...."
1247,33155_summary.txt,33155_2.txt,this is where we are differentiated by our emb...,HIGHLIGHTS\n2015 overview\nPortfolio value\n(...,"{'rouge-1': {'r': 0.2571428571428571, 'p': 0.0..."
1248,33155_summary.txt,33155_3.txt,this is where we are differentiated by our emb...,Setting Hammerson’s \nvalues and standards\nC...,"{'rouge-1': {'r': 0.1827956989247312, 'p': 0.1..."


**Rouge scores of all the 363 files on their golden summaries**

In [ ]:
rouge_scores = result_df['rouge_scores']
rouge_scores

0       {'rouge-1': {'r': 0.1966205837173579, 'p': 0.5...
1       {'rouge-1': {'r': 0.2653061224489796, 'p': 0.2...
2       {'rouge-1': {'r': 0.22871046228710462, 'p': 0....
3       {'rouge-1': {'r': 0.09648466716529544, 'p': 0....
4       {'rouge-1': {'r': 0.18396226415094338, 'p': 0....
                              ...                        
1245    {'rouge-1': {'r': 0.2727272727272727, 'p': 0.2...
1246    {'rouge-1': {'r': 0.18683001531393567, 'p': 0....
1247    {'rouge-1': {'r': 0.2571428571428571, 'p': 0.0...
1248    {'rouge-1': {'r': 0.1827956989247312, 'p': 0.1...
1249    {'rouge-1': {'r': 0.08823529411764706, 'p': 0....
Name: rouge_scores, Length: 1250, dtype: object

**Averaging out the rouge scores**

In [ ]:
# Initialize accumulated scores
total_rouge_1 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_2 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_l = {'r': 0, 'p': 0, 'f': 0}

# Accumulate scores
for scores_dict in rouge_scores:
    total_rouge_1['r'] += scores_dict['rouge-1']['r']
    total_rouge_1['p'] += scores_dict['rouge-1']['p']
    total_rouge_1['f'] += scores_dict['rouge-1']['f']

    total_rouge_2['r'] += scores_dict['rouge-2']['r']
    total_rouge_2['p'] += scores_dict['rouge-2']['p']
    total_rouge_2['f'] += scores_dict['rouge-2']['f']

    total_rouge_l['r'] += scores_dict['rouge-l']['r']
    total_rouge_l['p'] += scores_dict['rouge-l']['p']
    total_rouge_l['f'] += scores_dict['rouge-l']['f']

# Calculate average scores
num_scores = len(rouge_scores)

avg_rouge_1 = {key: total_rouge_1[key] / num_scores for key in total_rouge_1}
avg_rouge_2 = {key: total_rouge_2[key] / num_scores for key in total_rouge_2}
avg_rouge_l = {key: total_rouge_l[key] / num_scores for key in total_rouge_l}

print("Average ROUGE-1 scores:", avg_rouge_1)
print("Average ROUGE-2 scores:", avg_rouge_2)
print("Average ROUGE-L scores:", avg_rouge_l)

Average ROUGE-1 scores: {'r': 0.21635777895086972, 'p': 0.39328076932497735, 'f': 0.25599329811720284}
Average ROUGE-2 scores: {'r': 0.06978145064886791, 'p': 0.17662227939074815, 'f': 0.09073286454721821}
Average ROUGE-L scores: {'r': 0.20107794201046614, 'p': 0.3683948877585349, 'f': 0.23843035156735304}


**Function to calculate the BERT Score**

In [ ]:
# !pip install bert_score
import torch

In [ ]:
from bert_score import score

Pre = []
Rec = []
F1S = []

def calculate_bert(row):
    referenced_summary = row['gold_summary']
    generated_summary = row['summary']
    reference_sentences = referenced_summary.split('. ')
    generated_sentences = generated_summary.split('. ')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    min_length = min(len(generated_sentences), len(reference_sentences))
    reference_sentences = reference_sentences[:min_length]
    generated_sentences = generated_sentences[:min_length]
    P, R, F1 = score(generated_sentences, reference_sentences, lang='en', verbose=True, device=device)
    Pre.append(P.mean().item())
    Rec.append(R.mean().item())
    F1S.append(F1.mean().item())
    return F1.mean().item()

result_df['BERT F1 Score'] = result_df.apply(calculate_bert, axis=1)

done in 0.21 seconds, 23.88 sentences/sec


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.52 seconds, 5.74 sentences/sec


In [ ]:
result_df

,generated_summary_file,gold_summary_file,summary,gold_summary,BERT F1 Score
0,30777_summary.txt,30777_1.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\n02...,0.816831
1,30777_summary.txt,30777_2.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nFi...,0.815280
2,30777_summary.txt,30777_3.txt,bodycote lives by culture of honest and trans...,25695 19 March 2018 3:29 PM Proof 7\nA....,0.824584
3,30778_summary.txt,30778_1.txt,jobs. company overview staffline is leading w...,10\nStaffline Group plc Annual Report 2017\nC...,0.807432
4,30778_summary.txt,30778_2.txt,jobs. company overview staffline is leading w...,3 Overview Strategic Report Corporate Governa...,0.790513
...,...,...,...,...,...
1245,33154_summary.txt,33154_3.txt,"we have set ourselves ambitious goals, not fi...",Dixons Carphone plc Annual Report and Accoun...,0.818216
1246,33155_summary.txt,33155_1.txt,this is where we are differentiated by our emb...,"IN CONVERSATION WITH \nDAVID ATKINS, CHIEF EX...",0.809993
1247,33155_summary.txt,33155_2.txt,this is where we are differentiated by our emb...,HIGHLIGHTS\n2015 overview\nPortfolio value\n(...,0.776878
1248,33155_summary.txt,33155_3.txt,this is where we are differentiated by our emb...,Setting Hammerson’s \nvalues and standards\nC...,0.810939


Display the mean of the BERT F1 score

In [ ]:
result_df['BERT F1 Score'].mean()

0.7981799041509628

In [ ]:
Pre

[0.821881115436554,
 0.8329045176506042,
 0.8322282433509827,
 0.8189701437950134,
 0.8212555646896362,
 0.8172430396080017,
 0.8197250366210938,
 0.824357271194458,
 0.8152371644973755,
 0.8343404531478882,
 0.8030020594596863,
 0.837575376033783,
 0.8059553503990173,
 0.8480844497680664,
 0.8214631080627441,
 0.8183671832084656,
 0.815700888633728,
 0.7864737510681152,
 0.7638657093048096,
 0.7934440970420837,
 0.8334771990776062,
 0.8314933776855469,
 0.8325164318084717,
 0.8285528421401978,
 0.825473964214325,
 0.7943869233131409,
 0.8316283226013184,
 0.8529664278030396,
 0.8130295276641846,
 0.80100017786026,
 0.8114868998527527,
 0.8111578226089478,
 0.8161373138427734,
 0.8153709769248962,
 0.8096244931221008,
 0.8161455392837524,
 0.8169198036193848,
 0.8167834877967834,
 0.826481819152832,
 0.810325026512146,
 0.8241978883743286,
 0.8159471154212952,
 0.8104737997055054,
 0.8050026893615723,
 0.8309028148651123,
 0.8146471977233887,
 0.8214658498764038,
 0.8154799938201904,
 

In [ ]:
print("The average BERT Score Precision is :" , sum(Pre) / len(Pre))
print("The average BERT Score Recall is :" , sum(Rec) / len(Rec))
print("The average BERT Score F1 Score is :" , sum(F1S) / len(F1S))

The average BERT Score Precision is : 0.8136325525522232
The average BERT Score Recall is : 0.7843043150186538
The average BERT Score F1 Score is : 0.7981799041509628


In [ ]:
import pandas as pd
df = pd.read_csv('results.csv')

# Filter data based on ROUGE types
rouge_1_data = df[df['ROUGE-Type'].str.contains('ROUGE-1')]
rouge_2_data = df[df['ROUGE-Type'].str.contains('ROUGE-2')]
rouge_l_data = df[df['ROUGE-Type'].str.contains('ROUGE-L')]
rouge_su_data = df[df['ROUGE-Type'].str.contains('ROUGE-SU')]

avg_rouge_1 = rouge_1_data['Avg_F-Score'].mean()
avg_rouge_2 = rouge_2_data['Avg_F-Score'].mean()
avg_rouge_l = rouge_l_data['Avg_F-Score'].mean()
avg_rouge_su = rouge_su_data['Avg_F-Score'].mean()

# Print the results
print(f'Average ROUGE-1 F-Score: {avg_rouge_1:.5f}')
print(f'Average ROUGE-2 F-Score: {avg_rouge_2:.5f}')
print(f'Average ROUGE-L F-Score: {avg_rouge_l:.5f}')
print(f'Average ROUGE-SU F-Score: {avg_rouge_su:.5f}')

Average ROUGE-1 F-Score: 0.35529
Average ROUGE-2 F-Score: 0.16456
Average ROUGE-L F-Score: 0.34831
Average ROUGE-SU F-Score: 0.20985
